# Hybrid Search with Gemini

This recipe will show you how to run hybrid search with embeddings from Google's Gemini.

In [ ]:
!pip install --q weaviate-client

In [2]:
!pip show weaviate-client

Name: weaviate-client
Version: 4.10.4
Summary: A python native Weaviate client
Home-page: https://github.com/weaviate/weaviate-python-client
Author: Weaviate
Author-email: hello@weaviate.io,
License: BSD 3-clause
Location: /usr/local/lib/python3.11/site-packages
Requires: authlib, grpcio, grpcio-health-checking, grpcio-tools, httpx, pydantic, validators
Required-by: crewai-tools, goldenverba, langchain-weaviate, llama-index-vector-stores-weaviate, vector-etl


In [3]:
import weaviate, os
from weaviate.embedded import EmbeddedOptions
import weaviate.classes as wvc
import weaviate.classes.config as wc
import requests, json
import weaviate.classes.query as wq

## Requirements

1. Weaviate cluster
    1. You can create a 14-day free sandbox on [WCD](https://console.weaviate.cloud/)
    2. [Embedded Weaviate](https://weaviate.io/developers/weaviate/installation/embedded)
    3. [Local deployment](https://weaviate.io/developers/weaviate/installation/docker-compose#starter-docker-compose-file)
    4. [Other options](https://weaviate.io/developers/weaviate/installation)
2. [Gemini API Key](https://aistudio.google.com/app/apike) or[ Vertex AI token](https://cloud.google.com/vertex-ai?hl=e)

## Connect to Weaviate

Only choose one option from the below.

You can access Gemini through Vertex AI or AI Studio. Depending on which you choose, you will need to change the header when connecting to Weaviate (`X-Goog-Vertex-Api-Key` or `X-Goog-Studio-Api-Key`)

**Weaviate Cloud Deployment**

In [4]:
WCD_URL = os.environ["WEAVIATE_URL"] # Replace with your Weaviate cluster URL
WCD_AUTH_KEY = os.environ["WEAVIATE_AUTH"] # Replace with your cluster auth key
GEMINI_KEY = os.environ["MISTRAL_API_KEY"] # Either or. Replace with your Gemini API key (AI Studio)
VERTEX_AI_KEY = os.environ["VERTEX_AI_KEY"] # Either or. Replace with your Vertex AI auth token


# Weaviate Cloud Deployment
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WCD_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WCD_AUTH_KEY),
      headers={ "X-Goog-Studio-Api-Key": GEMINI_KEY} # switch to `X-Goog-Vertex-Api-Key` if you're using Vertex AI
)

print(client.is_ready())

True


**Local Instance**

In [ ]:
import weaviate
from weaviate import WeaviateClient

def connect_to_weaviate(palm_key) -> WeaviateClient:
    # Connect to your local Weaviate instance deployed with Docker
    client = weaviate.connect_to_local(
        headers={
            "X-Goog-Vertex-Api-Key": vertex_ai_key,
        }
    )

    print(client.is_ready())

    return client

### Expired Google Cloud Token

If you're using Vertex AI, you will need to refresh your token every 60 minutes. 

**If you're using AI Studio, ignore the below portion.**

The Google Cloud's OAuth 2.0 access tokens only have a **one** hour lifetime. This means you have to replace the expired token with a valid one and it to Weaviate by re-instantiating the client. 

#### Option 1: With Google Cloud CLI

In [ ]:
import subprocess

def refresh_token_with_GC_CLI() -> str:
    result = subprocess.run(["gcloud", "auth", "print-access-token"], capture_output=True, text=True)
    if result.returncode != 0:
        print(f"Error refreshing token: {result.stderr}")
        return None
    return result.stdout.strip()

Then you could run the below cell periodically.

In [ ]:
# Run every 60 minutes
token = refresh_token_with_GC_CLI
client = connect_to_weaviate(token)

#### Option 2: With `google-auth`

See the links to google-auth in [Python](https://google-auth.readthedocs.io/en/master/index.html) and [Node.js](https://cloud.google.com/nodejs/docs/reference/google-auth-library/latest) libraries.

In [ ]:
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials

def get_credentials() -> Credentials:
    credentials = Credentials.from_service_account_file('path/to/your/service-account.json', scopes=['openid'])
    request = Request()
    credentials.refresh(request)
    return credentials

Then run the below periodically:

In [ ]:
# Run every 60 minutes
credentials = get_credentials()
client = connect_to_weaviate(credentials.token)

## Create a collection
> Collection stores your data and vector embeddings.

In [6]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "JeopardyQuestion", and then you need to re-import it again.

# Delete the collection if it already exists
if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyQuestion",

    vectorizer_config=wc.Configure.Vectorizer.text2vec_google_aistudio( # specify the vectorizer and model type you're using
        model_id="embedding-001" # optional. defaults to "embedding-001"
    ),

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="Question", data_type=wc.DataType.TEXT), 
        wc.Property(name="Answer", data_type=wc.DataType.TEXT),
        wc.Property(name="Category", data_type=wc.DataType.TEXT, skip_vectorization=True), 
    ]
)

print("Successfully created collection: JeopardyQuestion.")

Successfully created collection: JeopardyQuestion.


## Import Data

In [7]:
import requests, json
url = 'https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

# Insert data objects
response = jeopardy.data.insert_many(data)

# Note, the `data` array contains 10 objects, which is great to call insert_many with.
# However, if you have a milion objects to insert, then you should spit them into smaller batches (i.e. 100-1000 per insert)

if (response.has_errors):
    print(response.errors)
else:
    print("Insert complete.")

Insert complete.


In [8]:
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.aggregate.over_all(total_count=True)

print(response.total_count)

10


## Hybrid Search

The `alpha` parameter determines the weight given to the sparse and dense search methods. `alpha = 0` is pure sparse (bm25) search, whereas `alpha = 1` is pure dense (vector) search. 

Alpha is an optional parameter. The default is set to `0.75`.

### Hybrid Search only

The below query is finding Jeopardy questions about animals and is limiting the output to only two results. Notice `alpha` is set to `0.80`, which means it is weighing the vector search results more than bm25. If you were to set `alpha = 0.25`, you would get different results. 

In [9]:
# note, you can reuse the collection object from the previous cell.
# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

response = jeopardy.query.hybrid(
    query="northern beast",
    alpha=0.8,
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

ID: d2d97c7d-4ae6-44f3-9c50-57d84ee75699
Data: {
  "answer": "species",
  "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification",
  "category": "SCIENCE"
} 

ID: 0b488aa7-8e47-47ee-9362-8614dbbd919f
Data: {
  "answer": "the diamondback rattler",
  "question": "Heaviest of all poisonous snakes is this North American rattlesnake",
  "category": "ANIMALS"
} 

ID: 014219e8-03d4-4218-bfc5-37e237259353
Data: {
  "answer": "Antelope",
  "question": "Weighing around a ton, the eland is the largest species of this animal in Africa",
  "category": "ANIMALS"
} 



### Hybrid Search on a specific property

The `properties` parameter allows you to list the properties that you want bm25 to search on.

In [10]:
response = jeopardy.query.hybrid(
    query="northern beast",
    query_properties=["question"],
    alpha=0.8,
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

ID: d2d97c7d-4ae6-44f3-9c50-57d84ee75699
Data: {
  "answer": "species",
  "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification",
  "category": "SCIENCE"
} 

ID: 0b488aa7-8e47-47ee-9362-8614dbbd919f
Data: {
  "answer": "the diamondback rattler",
  "question": "Heaviest of all poisonous snakes is this North American rattlesnake",
  "category": "ANIMALS"
} 

ID: 014219e8-03d4-4218-bfc5-37e237259353
Data: {
  "answer": "Antelope",
  "question": "Weighing around a ton, the eland is the largest species of this animal in Africa",
  "category": "ANIMALS"
} 



### Hybrid Search with a `where` filter

Find Jeopardy questions about elephants, where the category is set to Animals.

In [11]:
import weaviate.classes.query as wq # wq is an alias to save us from typing weaviate.classes everywhere ;)

response = jeopardy.query.hybrid(
    query="northern beast",
    alpha=0.8,
    filters=wq.Filter.by_property("category").equal("Animals"),
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

ID: 0b488aa7-8e47-47ee-9362-8614dbbd919f
Data: {
  "answer": "the diamondback rattler",
  "question": "Heaviest of all poisonous snakes is this North American rattlesnake",
  "category": "ANIMALS"
} 

ID: 014219e8-03d4-4218-bfc5-37e237259353
Data: {
  "answer": "Antelope",
  "question": "Weighing around a ton, the eland is the largest species of this animal in Africa",
  "category": "ANIMALS"
} 

ID: 7ce9505e-d0bf-4620-981a-63811f3320a3
Data: {
  "answer": "the nose or snout",
  "question": "The gavial looks very much like a crocodile except for this bodily feature",
  "category": "ANIMALS"
} 



### Hybrid Search with a custom vector

You can pass in your own vector as input into the hybrid query, by using the `vector` parameter. 

In [ ]:
vector = [-0.0125526935, -0.021168863, ...] # paste an embedding here

response = jeopardy.query.hybrid(
    query="animal",
    vector=vector,
    limit=2
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")